<a href="https://colab.research.google.com/github/chaerui7967/stock_predict_news_and_youtube/blob/master/youtube_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import numpy as np
import datetime

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
def set_data(df,com_num = 0):  # panda, numpy, datetime, FinanceDataReader, konlpy, Counter
    # 일, 시간 나누기
    df['date'] = df['date'].astype(str)
    df['date_d'] = df['date'].str[:-2]
    df['date_h'] = df['date'].str[-2:]
    # 타입을 데이트 타입으로 만듬
    df['date_d'] = pd.to_datetime(df['date_d'])
    
    df = df.sort_values(by='date_d') # 일기준으로 오름차순 정렬
    
    # 널값 제거
    df = df.dropna()
    
    # 주말 및 공휴일 데이터
    ### Holidays
    try:
        # 서버가 열려있을 때
        db = pymysql.connect(user='root',
                             passwd='1234',
                             host='3.35.70.166',
                             db='proj',
                             charset='utf8')

        cursor = db.cursor(pymysql.cursors.DictCursor)

        sql = "select * from holidays"
        cursor.execute(sql)
        result = cursor.fetchall()
    
        # DataFrame으로 변경
        holi = pd.DataFrame(result)
        # db 닫기 --> 안하면 메모리 잡아먹음
        db.close()
    except:
        # 서버 없을 때 깃허브에서 바로 가져옴
        db_holi = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/Sentiment_Analysis/data/holidays.csv'
        holi = pd.read_csv(db_holi)
    
    # date 컬럼을 날짜 형식으로 변경
    holi['date'] = pd.to_datetime(holi['date'])
    
    
    ### ===================주말 및 공휴일 제외
    
    ## 주말 및 공휴일만 추출
    market_closed = holi[holi['holiday']=="O"].reset_index(drop=True)
    
    ## 휴장일 List 생성
    market_closed_list = list(market_closed['date'])
    
    # 주말 및 공휴일 제외
    while len(df[df['date_d'].isin(market_closed_list)]['date_d']) !=0:
        for date in df[df['date_d'].isin(market_closed_list)]['date_d'].index:
            df['date_d'][date] += datetime.timedelta(1)
    
    return df

In [42]:
youtube_df = pd.read_csv('/content/drive/My Drive/Final PJT - 업빛투/감성분석/data/youtube_total.csv')

In [43]:
youtube_df = set_data(youtube_df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [44]:
youtube_df.drop(['date','length', 'url', 'date_h','description'], 1, inplace=True)

In [45]:
youtube_df = youtube_df[['st_n', 'st_cd', 'ch_nm', 'date_d', 'title', 'text', 'views']]

In [46]:
youtube_df.columns = ['st_n', 'st_cd', 'ch_nm', 'date', 'title', 'text', 'views']

In [47]:
youtube_df.head()

,st_n,st_cd,ch_nm,date,title,text,views
1362,삼성전자,5930,한국경제TV,2012-06-26,[마감시황] 삼성전자 `흔들` 코스피도 `휘청`‥1820선 하락마감,매우 늘 증시 마감 상황 정리 해보겠습니다 증권 팀 지수 이 기자나와있습니다네 오늘...,30
60,하이닉스,660,한국경제TV,2012-07-02,"SK하이닉스, M12라인 가동 차세대 메모리 본격 강화",sk 하이닉스가 차세대 메모리 반도체 사업 확장에 본격적으로 돌입했습니다청주에 새로...,154
2016,LG화학,51910,한국경제TV,2012-07-14,6인의 마감전략 성공투자 오후증시2012년 07월 13일 방송,tweet4 7월 13일 금 일 장 마감 42분 전 이군요 자 6명의 전문가는어떤 ...,115
1283,삼성전자,5930,한국경제TV,2012-07-30,삼성전자 또 사상최대 실적 2분기 영업이익 6조7천억원,소식입니다 삼성전자가 또다시 사상 최대 실적을 경신했습니다 갤럭시 시리즈의 위력을 ...,36
1259,삼성전자,5930,한국경제TV,2012-09-13,"삼성전자, 반도체 중국생산 시대 연다",삼성전자가 중국 산시성 시안 시에서 낸드플래시 반도체 공장 기공식을열었습니다삼성전자...,164


In [48]:
youtube_df.to_csv('./youtube_preprocessing.csv', index=False)